In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [2]:
f= open('../datasets/shakespeare.txt','r')
d=str(f.read())
f.close()

In [3]:
data= d.strip().lower()
data=data[104:]

In [10]:
print data[5000:5100]

 a refund of the money (if any) you paid
for it by sending an explanatory note within that time to 


In [5]:
chars=sorted(list(set(data)))
print len(chars)

66


In [7]:
char_indices=dict((c,i) for i,c in enumerate(chars))
indices_char=dict((i,c) for i,c in enumerate(chars))
print len(char_indices),len(indices_char)

66 66


In [14]:
print data.split('\n')[1000:1010]

['  and scarcely greet me with that sun thine eye,\r', '  when love converted from the thing it was\r', '  shall reasons find of settled gravity;\r', '  against that time do i ensconce me here\r', '  within the knowledge of mine own desert,\r', '  and this my hand, against my self uprear,\r', '  to guard the lawful reasons on thy part,\r', '    to leave poor me, thou hast the strength of laws,\r', '    since why to love, i can allege no cause.\r', '\r']
